In [1]:
%matplotlib notebook 

import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import astropy.units as u
import astropy.constants as const

from scipy.interpolate import interp1d



mm_c = 12.01*const.u.value*const.u.unit
mm_co = 28.01*const.u.value*const.u.unit
mm_h2 = 2*const.u.value*const.u.unit

boltz_const = const.k_B.value * const.k_B.unit
grav_const = const.G.value * const.G.unit
stellar_mass = 1.76 * const.M_sun

h2_co_ratio_ism = 1e4
h2_co_ratio_pp1 = 1e5
h2_co_ratio_pp2 = 1e6

ring_mid = 7.5 * u.AU
ring_width = 3.3* u.AU


high_temperature = 169 * u.K
co_mass_lower_ht = (4.5e-6 * (const.M_earth).value)*(const.M_earth).unit
co_mass_upper_ht = (12.1e-6 * (const.M_earth).value)*(const.M_earth).unit

h2_mass_lower_ht_ism = (mm_h2/mm_co) * h2_co_ratio_ism * co_mass_lower_ht
h2_mass_upper_ht_ism = (mm_h2/mm_co) * h2_co_ratio_ism * co_mass_upper_ht

low_temperature = 50 * u.K
co_mass_lt = (0.01 * (const.M_earth).value)*(const.M_earth).unit

unshield_timescale = 1*u.day


In [9]:
def find_nearest(array,value):
    """
    Inputs:
        -   array is the array you're searching for a value in
        -   value is the value you're searching for
    Returns:
        -   idx, the index of the array entry closest to the value
    """
    temp = 1000
    for i in range(0,len(array)):
        if  np.abs(array[i]-value) <= temp:
            temp = np.abs(array[i]-value)
            idx = i
    return(idx)

def calc_column_density(mass, temperature, mm):
    """
    Inputs: 
        - mass:         mass of the species in question
        - temperature:  temperature locally
        - mm:           molar mass of the dominating species
    Output: 
        - column_density: in 1/cm^2
        
    Notes:
        - aspect ratio is the ratio of the sound speed, c_s and keplerian velocity, v_k
          It is the ratio of the height of the disk to the radial location
    """
    
    aspect_ratio = math.sqrt((boltz_const * temperature * ring_mid) / (mm * grav_const * stellar_mass))
    path_length = (2 * math.sqrt((2 * ring_mid * ring_width).value) * u.AU).to(u.cm)
    
    ring_volume = (2 * math.pi * aspect_ratio * ring_width * ring_mid**2).to(u.cm**3)
    
    column_density = (mass * path_length) / (mm * ring_volume)
    
    return(column_density)

def calc_mass(column_density, temperature, mm):
    """
    Inputs: 
        - column_density: column density of the species in question
        - temperature:    temperature locally
        - mm:             molar mass of the dominating species
    Outputs: 
        - mass:   in kg
        
    Notes:
        - aspect ratio is the ratio of the sound speed, c_s and keplerian velocity, v_k
          It is the ratio of the height of the disk to the radial location        
    """
    aspect_ratio = math.sqrt((boltz_const * temperature * ring_mid) / (mm * grav_const * stellar_mass))
    path_length = (2 * math.sqrt((2 * ring_mid * ring_width).value) * u.AU).to(u.cm)
    
    ring_volume = (2 * math.pi * aspect_ratio * ring_width * ring_mid**2).to(u.cm**3)
    
    mass = column_density * mm * ring_volume / path_length
    return(mass)

In [3]:
colden_co_lower_ht = calc_column_density(co_mass_lower_ht, high_temperature, mm_co)
colden_co_upper_ht = calc_column_density(co_mass_upper_ht, high_temperature, mm_co)

colden_h2_lower_ht_ism = calc_column_density(h2_mass_lower_ht_ism, high_temperature, mm_h2)
colden_h2_upper_ht_ism = calc_column_density(h2_mass_upper_ht_ism, high_temperature, mm_h2)

colden_co_lt = calc_column_density(co_mass_lt, low_temperature, mm_co)


In [4]:
filepath = "./co_values/"

labels = ["4000K",
          "10000K",
          "20000K",
          "solar"]

filenames = ["photodissociation_4000K_shielding.txt",
             "photodissociation_10000K_shielding.txt",
             "photodissociation_20000K_shielding.txt",
             "photodissociation_solar_shielding.txt"]

shielding_funcs = {}
for index in range(0,len(filenames)):
    temp = np.loadtxt(filepath+filenames[index],skiprows = 4, dtype=str)
    temp = np.float_(temp)
    shielding_funcs.update({labels[index]:temp})
    
axes_info = {"h2":{"dat_column":1,
                   "title":"CO Shielding by H2"},
             "self":{"dat_column":3,
                     "title":"CO self-shielding"},
             "c":{"dat_column":4,
                  "title":"CO Shielding by C"},
             "co":{"dat_column":6,
                   "title":"CO Shielding by CO"}}


If you want to evaluate the interpolated shielding functions, the appropriate value will be 10**shield_func(np.log10(value))


In [5]:
selfshield_func_10000K = interp1d(np.log10(shielding_funcs['10000K'][:,0]),np.log10(shielding_funcs['10000K'][:,3]))
selfshield_func_4000K = interp1d(np.log10(shielding_funcs['4000K'][:,0]),np.log10(shielding_funcs['4000K'][:,3]))

h2shield_func_10000K = interp1d(np.log10(shielding_funcs['10000K'][:,0]),np.log10(shielding_funcs['10000K'][:,1]))
h2shield_func_4000K = interp1d(np.log10(shielding_funcs['4000K'][:,0]),np.log10(shielding_funcs['4000K'][:,1]))

cshield_func_10000K = interp1d(np.log10(shielding_funcs['10000K'][:,0]),np.log10(shielding_funcs['10000K'][:,4]))
cshield_func_4000K = interp1d(np.log10(shielding_funcs['4000K'][:,0]),np.log10(shielding_funcs['4000K'][:,4]))

colden_for_interp = np.logspace(13,24,100000)*(1/u.cm**2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log10
  


Below, we calculate the self-shielding present for three masses of CO in the system:

    1. The mass of CO derived for a low temp case (<50 K), where the CO is optically thick. 
    2. The upper-bound mass of CO for a high temper case (>50 K), where the CO is optically thin. 
    3. The lower-bound mass of CO for a high temp case (>50 K), where the CO is optically thin

In [6]:
shield_4000K = 10**selfshield_func_4000K(np.log10(colden_co_lower_ht.value))
shield_10000K = 10**selfshield_func_10000K(np.log10(colden_co_lower_ht.value))
shield_8000K_lower_ht = (shield_4000K+(2/3)*(shield_10000K - shield_4000K))

shield_4000K = 10**selfshield_func_4000K(np.log10(colden_co_upper_ht.value))
shield_10000K = 10**selfshield_func_10000K(np.log10(colden_co_upper_ht.value))
shield_8000K_upper_ht = (shield_4000K+(2/3)*(shield_10000K - shield_4000K))

shield_4000K = 10**selfshield_func_4000K(np.log10(colden_co_lt.value))
shield_10000K = 10**selfshield_func_10000K(np.log10(colden_co_lt.value))
shield_8000K_lt = (shield_4000K+(2/3)*(shield_10000K - shield_4000K))

print(f"For a CO mass of {co_mass_lower_ht:.2e}, the CO column density is {colden_co_lower_ht:.2e}")
print(f"\t This yields a CO lifetime of {(unshield_timescale/shield_8000K_lower_ht).to(u.yr):.2e}")
print(f"For a CO mass of {co_mass_upper_ht:.2e}, the CO column density is {colden_co_upper_ht:.2e}")
print(f"\t This yields a CO lifetime of {(unshield_timescale/shield_8000K_upper_ht).to(u.yr):.2e}")
print(f"For a CO mass of {co_mass_lt:.2e}, the CO column density is {colden_co_lt:.2e}")
print(f"\t This yields a CO lifetime of {(unshield_timescale/shield_8000K_lt).to(u.yr):.2e}")


For a CO mass of 2.69e+19 kg, the CO column density is 2.01e+18 1 / cm2
	 This yields a CO lifetime of 1.83e+00 yr
For a CO mass of 7.23e+19 kg, the CO column density is 5.40e+18 1 / cm2
	 This yields a CO lifetime of 4.11e+00 yr
For a CO mass of 5.97e+22 kg, the CO column density is 8.20e+21 1 / cm2
	 This yields a CO lifetime of 3.08e+04 yr


Below, we calculate the shielding from H2 if H2 was present at the interstellar CO/H2 ratio of 10^-4 and update the aspect ratio (and thus ring volume) to one where h2 is the dominant species, and thus dominates the scale height. 

In [7]:
shield_4000K = 10**h2shield_func_4000K(np.log10(colden_h2_lower_ht_ism.value*10))
shield_10000K = 10**h2shield_func_10000K(np.log10(colden_h2_lower_ht_ism.value*10))
shield_8000K_lower_ht = (shield_4000K+(2/3)*(shield_10000K - shield_4000K))

shield_4000K = 10**h2shield_func_4000K(np.log10(colden_h2_upper_ht_ism.value*10))
shield_10000K = 10**h2shield_func_10000K(np.log10(colden_h2_upper_ht_ism.value*10))
shield_8000K_upper_ht = (shield_4000K+(2/3)*(shield_10000K - shield_4000K))

print(f"For a CO mass of {co_mass_lower_ht:.2e} and H2/CO ISM ratio of {h2_co_ratio_ism:.2e}")
print(f"\t the H2 mass is {h2_mass_lower_ht_ism:.2e} and the H2 column density is {colden_h2_lower_ht_ism:.2e}")
print(f"\t\t This yields a CO lifetime of {(unshield_timescale/shield_8000K_lower_ht).to(u.yr):.2e}")

print(f"For a CO mass of {co_mass_upper_ht:.2e} and H2/CO ISM ratio of {h2_co_ratio_ism:.2e}")
print(f"\t the H2 mass is {h2_mass_upper_ht_ism:.2e}  and the H2 column density is {colden_h2_upper_ht_ism:.2e}")
print(f"\t\t This yields a CO lifetime of {(unshield_timescale/shield_8000K_upper_ht).to(u.yr):.2e}")

For a CO mass of 2.69e+19 kg and H2/CO ISM ratio of 1.00e+04
	 the H2 mass is 1.92e+22 kg and the H2 column density is 5.36e+21 1 / cm2
		 This yields a CO lifetime of 2.15e+04 yr
For a CO mass of 7.23e+19 kg and H2/CO ISM ratio of 1.00e+04
	 the H2 mass is 5.16e+22 kg  and the H2 column density is 1.44e+22 1 / cm2
		 This yields a CO lifetime of 1.94e+10 yr


Below, we calculate the mass of carbon required to shield the CO for 0.2 Myr. This timescale is chosen as it is the length of time needed for debris to circularize after a giant impact in this system at this age. 

In [10]:
interp_cshield_func_10000K = 10**cshield_func_10000K(np.log10(colden_for_interp.value))
interp_cshield_func_4000K = 10**cshield_func_4000K(np.log10(colden_for_interp.value))

timescale_pt2Myr = (0.2e6 * u.yr).to(unshield_timescale.unit)
shield_target = unshield_timescale/timescale_pt2Myr 

idx_10000K_pt2Myr = find_nearest(interp_cshield_func_10000K,shield_target.value)
idx_4000K_pt2Myr = find_nearest(interp_cshield_func_4000K,shield_target.value)

colden_10000K_pt2Myr = colden_for_interp[idx_10000K_pt2Myr]
colden_4000K_pt2Myr = colden_for_interp[idx_4000K_pt2Myr]
colden_8000K_pt2Myr = colden_4000K_pt2Myr + \
                      (2/3)*(colden_10000K_pt2Myr - colden_4000K_pt2Myr)

cmass_8000K_pt2Myr = calc_mass(colden_8000K_pt2Myr, high_temperature, mm_c)

print(f"To achieve a lifetime of {timescale_pt2Myr.to(u.yr):.1e}")
print("\tFor a 10000 K star, you need a C column density of ")
print(f"\t\t{colden_10000K_pt2Myr:.2e}")
print("\tFor a 4000 K star, you need a C column density of ")
print(f"\t\t{colden_4000K_pt2Myr:.2e}")
print("Assuming the column density can be scaled for an 8000K star, ")
print("\tFor a 8000 K star, you need a C column density of ")
print(f"\t\t{colden_8000K_pt2Myr:.2e}")
print("\tThis translates to a C mass of ")
print(f"\t\t{cmass_8000K_pt2Myr:.2e}")
print(f"\t\t{cmass_8000K_pt2Myr/const.M_earth:.2e} M_earth")
print(f"This yields C/CO ratios at 169 K ranging between")
print(f"\t{cmass_8000K_pt2Myr/co_mass_lower_ht:0.2e}")
print(f"\t{cmass_8000K_pt2Myr/co_mass_upper_ht:0.2e}")

To achieve a lifetime of 2.0e+05 yr
	For a 10000 K star, you need a C column density of 
		1.06e+18 1 / cm2
	For a 4000 K star, you need a C column density of 
		1.03e+18 1 / cm2
Assuming the column density can be scaled for an 8000K star, 
	For a 8000 K star, you need a C column density of 
		1.05e+18 1 / cm2
	This translates to a C mass of 
		9.17e+18 kg
		1.54e-06 M_earth
This yields C/CO ratios at 169 K ranging between
	3.41e-01
	1.27e-01


Below, we calculate the amount of carbon needed to shield the CO on a timescale of 23 Myr, or the age of the system. 

In [11]:
timescale_23Myr = (23e6 * u.yr).to(unshield_timescale.unit)
shield_target = unshield_timescale/timescale_23Myr

idx_10000K_23Myr = find_nearest(interp_cshield_func_10000K,shield_target.value)
idx_4000K_23Myr = find_nearest(interp_cshield_func_4000K,shield_target.value)

colden_10000K_23Myr = colden_for_interp[idx_10000K_23Myr]
colden_4000K_23Myr = colden_for_interp[idx_4000K_23Myr]
colden_8000K_23Myr = colden_4000K_23Myr + \
                     (2/3)*(colden_10000K_23Myr - colden_4000K_23Myr)

cmass_8000K_23Myr = calc_mass(colden_8000K_23Myr, high_temperature, mm_c)

print(f"To achieve a lifetime of {timescale_23Myr.to(u.yr):.1e}")
print("\tFor a 10000 K star, you need a C column density of ")
print(f"\t\t{colden_10000K_23Myr:.2e}")
print("\tFor a 4000 K star, you need a C column density of ")
print(f"\t\t{colden_4000K_23Myr:.2e}")
print("Assuming the column density can be scaled for an 8000K star, ")
print("\tFor a 8000 K star, you need a C column density of ")
print(f"\t\t{colden_8000K_23Myr:.2e}")
print("\tThis translates to a C mass of ")
print(f"\t\t{cmass_8000K_23Myr:.2e}")
print(f"\t\t{cmass_8000K_23Myr/const.M_earth:.2e} M_earth")
print(f"This yields CO/C ratios at 169 K ranging between")
print(f"\t{co_mass_lower_ht/cmass_8000K_23Myr:0.2e}")
print(f"\t{co_mass_upper_ht/cmass_8000K_23Myr:0.2e}")

To achieve a lifetime of 2.3e+07 yr
	For a 10000 K star, you need a C column density of 
		1.34e+18 1 / cm2
	For a 4000 K star, you need a C column density of 
		1.30e+18 1 / cm2
Assuming the column density can be scaled for an 8000K star, 
	For a 8000 K star, you need a C column density of 
		1.32e+18 1 / cm2
	This translates to a C mass of 
		1.16e+19 kg
		1.95e-06 M_earth
This yields CO/C ratios at 169 K ranging between
	2.31e+00
	6.22e+00
